## ExMAS
> Exact Matching of Attractive Shared rides (ExMAS) for system-wide strategic evaluations
> 
> Kucharski R., Cats O. ,_Transportation Research Part: B_, 2020

---

In [ ]:
import os
cwd = os.getcwd()

In [2]:
os.chdir(os.path.join(cwd,'../..'))
import ExMAS.main
import ExMAS.utils

In [3]:
from ExMAS.utils import inData as inData
from ExMAS.main import matching

In [4]:
import random
%load_ext autoreload
%autoreload 2

### prepare

1. __load the default config__ (see more details in `ExMAS/notebooks/01 configuration.ipynb`)

In [36]:
params = ExMAS.utils.get_config('ExMAS/data/configs/default.json') # load the default 
params.nP = 1000
params.simTime = 4

In [37]:
params.participation = 0.5
params.p = 0.8

2. __load the network__ (see more details in `ExMAS/notebooks/02 NetworkGraphs.ipynb`)

In [38]:
inData = ExMAS.utils.load_G(inData, params, stats=True)  # download the graph

3. __generate trip requests__ (see more details in `ExMAS/notebooks/02 NetworkGraphs.ipynb`)

In [43]:
inData = ExMAS.utils.generate_demand(inData, params)

In [31]:
params.logger_level = 'INFO'
params.matching_obj = 'u_veh'
params.without_matching = True
params.multi_platform_matching = True

In [32]:
import networkx as nx
import math
import pandas as pd

### run

In [44]:
inData = ExMAS.main(inData, params)

03-07-20 15:56:42-INFO-Initializing pairwise trip shareability between 1000 and 1000 trips.
03-07-20 15:56:42-INFO-creating combinations
03-07-20 15:56:43-INFO-999000	 nR*(nR-1)
03-07-20 15:56:57-INFO-Reduction of feasible pairs by 99.85%
03-07-20 15:56:57-INFO-Degree 2 	Completed
03-07-20 15:56:57-INFO-trips to extend at degree 2 : 4222
03-07-20 15:57:02-INFO-At degree 2 feasible extensions found out of 621 searched
03-07-20 15:57:02-INFO-Degree 3 	Completed
03-07-20 15:57:02-INFO-trips to extend at degree 3 : 621
03-07-20 15:57:03-INFO-At degree 3 feasible extensions found out of 35 searched
03-07-20 15:57:03-INFO-Degree 4 	Completed
03-07-20 15:57:03-INFO-trips to extend at degree 4 : 35
03-07-20 15:57:03-INFO-At degree 4 feasible extensions found out of 0 searched
03-07-20 15:57:03-INFO-Degree 5 	Completed
03-07-20 15:57:03-INFO-No more trips to exted at degree 5
03-07-20 15:57:07-INFO-Matching 1000 trips to 5878 rides in order to minimize u_veh
03-07-20 15:57:12-INFO-Problem solut

In [1]:
for p in [0.6, 0.8, 0.9]:
    params.p = p
    share_of_active = params.participation / params.p
    inData.passengers['active'] = False
    inData.passengers.active.loc[inData.requests.sample(int(share_of_active*params.nP)).index] = True
    for day in range(1,10):
        print(p, day)
        inData.passengers['active_today'] = inData.passengers.apply(lambda x : is_active_today(x), axis = 1)
        inData.passengers['platforms'] = inData.passengers.apply(lambda x : [0] if x.active_today else [-1], axis = 1)
        inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis=1)
        inData.sblts.requests['platform'] = inData.requests['platform']
        inData = matching(inData, params)        
        G = ExMAS.utils.make_graph(inData.sblts.requests[inData.sblts.requests.platform == 0], inData.sblts.schedule, mu = 0.5, plot = False, node_size = 1)
        nx.write_edgelist(G, 'ExMAS/spinoffs/edgelists/matching_p{}_day{}.csv'.format(params.p, day), delimiter = ',', data = False)
        G = ExMAS.utils.make_graph(inData.sblts.requests[inData.sblts.requests.platform == 0], inData.sblts.rides, mu = 0.5, plot = False, node_size = 1)
        nx.write_edgelist(G, 'ExMAS/spinoffs/edgelists/potential_p{}_day{}.csv'.format(params.p, day), delimiter = ',', data = False)

NameError: name 'params' is not defined